## 1. Load the Data

* First connect the drive to the Lab before any processes.
* Load all the available data that we have available to our project.
* It may impede the process and speed but I think reducing the bactch number will help.
* Splitting data into training, validation, and test sets.



In [1]:
import os
import random
import shutil
import numpy as np
import torch
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim
import torchvision

from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader, Dataset, WeightedRandomSampler
from collections import Counter
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'torch'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
root_folders = [
    "/content/drive/MyDrive/CoWorkStuff/Data/01_test_and_val_dataset",
    "/content/drive/MyDrive/CoWorkStuff/Data/02_test_and_val_dataset_improved",
    "/content/drive/MyDrive/CoWorkStuff/Data/03_test_and_val_dataset_resampled",
    "/content/drive/MyDrive/CoWorkStuff/Data/04_test_and_val_dataset_resampled_resized",
    "/content/drive/MyDrive/CoWorkStuff/Data/05_train_dataset",
    "/content/drive/MyDrive/CoWorkStuff/Data/06_train_deduplicated",
    "/content/drive/MyDrive/CoWorkStuff/Data/07_train_deduplicated_dataset",
    "/content/drive/MyDrive/CoWorkStuff/Data/08_train_data",
    "/content/drive/MyDrive/CoWorkStuff/Data/09_train_data_resized",
    "/content/drive/MyDrive/CoWorkStuff/Data/10_train_pal_data",
    "/content/drive/MyDrive/CoWorkStuff/Data/11_test_pal_data",
    "/content/drive/MyDrive/CoWorkStuff/Data/12_val_pal_data"
    ]


# Define a combined folder to store the merged data
combined_folder = "/content/drive/MyDrive/CoWorkStuff/CombinedData"

# Create the combined dataset folder if it doesn't exist
os.makedirs(combined_folder, exist_ok=True)

In [ ]:
# Define a dictionary to map subfolder names to class labels
class_mapping = {
    'subfolder1': 'Cercospora',
    'subfolder2': 'Healthy',
    'subfolder3': 'Miner',
    'subfolder4': 'Phoma',
    'subfolder5': 'Rust'
    }

# Move subfolder content into the combined folder
def move_subfolder_content(src, dst):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      os.makedirs(d, exist_ok=True)
      move_subfolder_content(s, d)
    else:
      shutil.move(s, d)

# Combine data from different root folders into the combined folder
for root_folder in root_folders:
  for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)
    if os.path.isdir(subfolder_path):
      # Get the class label from the dictionary
      class_label = class_mapping.get(subfolder, 'Other')
      # Create a target folder based on the class label
      target_folder = os.path.join(combined_folder, class_label)
      # Create the target folder if it doesn't exist
      os.makedirs(target_folder, exist_ok=True)
      # Move the subfolder content to the target folder
      move_subfolder_content(subfolder_path, target_folder)

In [ ]:
# Recursively collect image files from the folder(s)
def collect_image_files(folder):
  image_files = []
  for root, _, files in os.walk(folder):
    for file in files:
      if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        image_files.append(os.path.join(root, file))
  return image_files

# Shuffle and split data for each class
def shuffle_and_split_data(root_folder, target_root, train_ratio, val_ratio):
  for class_label in os.listdir(root_folder):
    class_folder = os.path.join(root_folder, class_label)

    if os.path.isdir(class_folder):
      # List all files in the class folder
      class_data = collect_image_files(class_folder)
      random.shuffle(class_data)

      # Split the data into train, validation, and test sets
      num_samples = len(class_data)
      num_train = int(train_ratio * num_samples)
      num_val = int(val_ratio * num_samples)

      train_data = class_data[:num_train]
      val_data = class_data[num_train:num_train + num_val]
      test_data = class_data[num_train + num_val:]

      # Create target folders
      train_folder = os.path.join(target_root, 'train', class_label)
      val_folder = os.path.join(target_root, 'val', class_label)
      test_folder = os.path.join(target_root, 'test', class_label)

      os.makedirs(train_folder, exist_ok=True)
      os.makedirs(val_folder, exist_ok=True)
      os.makedirs(test_folder, exist_ok=True)

      # Move data to respective folders
      move_data(train_data, class_folder, train_folder)
      move_data(val_data, class_folder, val_folder)
      move_data(test_data, class_folder, test_folder)

# Define move_data
def move_data(data_list, source_folder, dest_folder):
  for data in data_list:
    src_path = os.path.join(source_folder, data)
    dst_path = os.path.join(dest_folder, data)
    if os.path.exists(src_path):
      shutil.move(src_path, dst_path)

# Define the split ratios
train_ratio = 0.65
val_ratio = 0.15
test_ratio = 0.20

# Define the target root folder for the train, val, and test sets
target_root = "/content/drive/MyDrive/CoWorkStuff/SplitData"

# Create the train, validation, and test folders
os.makedirs(os.path.join(target_root, 'train'), exist_ok=True)
os.makedirs(os.path.join(target_root, 'val'), exist_ok=True)
os.makedirs(os.path.join(target_root, 'test'), exist_ok=True)

# Shuffle and split data for each class
shuffle_and_split_data(combined_folder, target_root, train_ratio, val_ratio)

#######################################################################################################

## 1. Load the **Models Pre-trained** by our talented model building team

* Select as many models that were created.
* If possible, check the best performing one or possibly fine tunning them.
* Preferrably ensamble the models to make it more solid and stronger.
* Without any particular order or favouritism, for the pre-trained models; credit, thanks and props goes out to the following collaborators:
  - Darshan
  - Lucas
  - Dimitra
  - Juan

In [ ]:
# Load .h5 model (TensorFlow/Keras)
model_01 = keras.models.load_model("/content/drive/MyDrive/CoWorkStuff/Models/model_CNN1_BRACOL.h5")
model_02 = keras.models.load_model("/content/drive/MyDrive/CoWorkStuff/Models/baseline_resnet50.h5")
model_03 = keras.models.load_model("/content/drive/MyDrive/CoWorkStuff/Models/without_cersc_and_healthy_resnet50_deduplicated_mix_val_train_67acc.h5")
model_04 = keras.models.load_model("/content/drive/MyDrive/CoWorkStuff/Models/without_cersc_and_healthy_resnet50_deduplicated.h5")
model_05 = keras.models.load_model("/content/drive/MyDrive/CoWorkStuff/Models/without_cersc_and_healthy_resnet50.h5")
model_06 = keras.models.load_model("/content/drive/MyDrive/CoWorkStuff/Models/without_cersc_resnet50_deduplicated_mix_val_train_75acc.h5")


In [ ]:
# Load .pth model (PyTorch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

class CoffeeLeafClassifier(nn.Module):
  def __init__(self):
    super(CoffeeLeafClassifier, self).__init__()

    # Convolutional layers
    self.conv_layers = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(64, 128, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        )
    # Fully connected layers
    self.fc_layers = nn.Sequential(
        nn.Linear(128 * 30 * 30, 512),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.5),

        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 5) # 5 classes
        )

  def forward(self, x):
    x = self.conv_layers(x)
    x = x.view(x.size(0), -1) # Flatten the output
    x = self.fc_layers(x)
    return x

model_path = os.path.join(
    "/content/drive/MyDrive/CoWorkStuff/Models/cnn_strategy1_weighted_loss",
    "/content/drive/MyDrive/CoWorkStuff/Models/cnn_strategy1_weighted_loss/coffee_leaf_classifier.pth")
model_07 = torch.load(model_path, map_location=torch.device('cpu'))
model_07.to(device)
model_07.eval()

cpu


CoffeeLeafClassifier(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=115200, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=128, out_features=5, bias=True)
  )
)

In [ ]:
# Load the MobileNetV2 model with random weights
model_08 = MobileNetV2(weights="imagenet", include_top=False)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import accuracy_score, classification_report
import cv2

# Define a function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size):
  img = load_img(image_path, target_size=target_size)
  img_array = img_to_array(img)
  img_array = preprocess_input(img_array)
  return img_array

# Define the folder containing your test data
test_data_folder = "/content/drive/MyDrive/CoWorkStuff/SplitData/test"

# Define lists to store model predictions
predictions_model_01 = []
predictions_model_02 = []
predictions_model_03 = []
predictions_model_04 = []
predictions_model_05 = []
predictions_model_06 = []
predictions_model_07 = []
predictions_model_08 = []

# Define a list to store true labels
true_labels = []

# Class Maping
class_mapping = {
    'Cercospora': 0,
    'Healthy': 1,
    'Miner': 2,
    'Phoma': 3,
    'Rust': 4
    }

# Iterate through the test data folder
for class_label in os.listdir(test_data_folder):
  class_folder = os.path.join(test_data_folder, class_label)
  true_class = class_mapping[class_label]

  # Iterate through the images in the class folder
  for image_name in os.listdir(class_folder):
    image_path = os.path.join(class_folder, image_name)

    # Load and preprocess the image
    img = load_and_preprocess_image(image_path, target_size=(256, 256))
    img_02 = load_and_preprocess_image(image_path, target_size=(224, 224))

    img_03 = cv2.imread(image_path)
    img_03 = cv2.resize(img_03, (256, 256)).astype(np.float32)
    img_03 = cv2.cvtColor(img_03, cv2.COLOR_BGR2RGB)
    img_03 = preprocess_input(img_03)
    img_03 = img_03 / 255.0

    # Store the true label
    true_labels.append(true_class)

    # Make predictions using each model
    pred_model_01 = model_01.predict(np.expand_dims(img, axis=0))
    pred_model_02 = model_02.predict(np.expand_dims(img_02, axis=0))
    pred_model_03 = model_03.predict(np.expand_dims(img_02, axis=0))
    pred_model_04 = model_04.predict(np.expand_dims(img_02, axis=0))
    pred_model_05 = model_05.predict(np.expand_dims(img_02, axis=0))
    pred_model_06 = model_06.predict(np.expand_dims(img_02, axis=0))
    pred_model_08 = model_08.predict(np.expand_dims(img, axis=0))

    img_tensor = torch.tensor(img_03.transpose(2, 0, 1)).unsqueeze(0)
    with torch.no_grad():
      pred_model_07 = model_07(img_tensor)

    # Append predictions to respective lists
    predictions_model_01.append(np.argmax(pred_model_01))
    predictions_model_02.append(np.argmax(pred_model_02))
    predictions_model_03.append(np.argmax(pred_model_03))
    predictions_model_04.append(np.argmax(pred_model_04))
    predictions_model_05.append(np.argmax(pred_model_05))
    predictions_model_06.append(np.argmax(pred_model_06))
    predictions_model_07.append(np.argmax(pred_model_07))
    predictions_model_08.append(np.argmax(pred_model_08))

# Evaluate the model predictions (e.g., accuracy, classification report)
accuracy_model_01 = accuracy_score(true_labels, predictions_model_01)
accuracy_model_02 = accuracy_score(true_labels, predictions_model_02)
accuracy_model_03 = accuracy_score(true_labels, predictions_model_03)
accuracy_model_04 = accuracy_score(true_labels, predictions_model_04)
accuracy_model_05 = accuracy_score(true_labels, predictions_model_05)
accuracy_model_06 = accuracy_score(true_labels, predictions_model_06)
accuracy_model_07 = accuracy_score(true_labels, predictions_model_07)
accuracy_model_08 = accuracy_score(true_labels, predictions_model_08)

print("Accuracy (Model_01):", accuracy_model_01)
print("Accuracy (Model_02):", accuracy_model_02)
print("Accuracy (Model_03):", accuracy_model_03)
print("Accuracy (Model_04):", accuracy_model_04)
print("Accuracy (Model_05):", accuracy_model_05)
print("Accuracy (Model_06):", accuracy_model_06)
print("Accuracy (Model_07):", accuracy_model_07)
print("Accuracy (Model_08):", accuracy_model_08)

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 0s 64ms/step


OSError: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import accuracy_score, classification_report
import cv2
import os
import numpy as np

# Define a function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size):
  img = cv2.imread(image_path)

  # Check if the image is read correctly, handle bad images
  if img is not None:
    img = cv2.resize(img, target_size).astype(np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = preprocess_input(img)
    img = img / 255.0
    return img
  else:
    return None

# Define the folder containing your test data
test_data_folder = "/content/drive/MyDrive/CoWorkStuff/SplitData/test"

# Define lists to store model predictions
predictions_model_01 = []
predictions_model_02 = []
predictions_model_03 = []
predictions_model_04 = []
predictions_model_05 = []
predictions_model_06 = []
predictions_model_07 = []
predictions_model_08 = []

# Define a list to store true labels
true_labels = []

# Define a list to store bad image paths
bad_file_list = []
bad_count = 0

# Class Mapping
class_mapping = {
    'Cercospora': 0,
    'Healthy': 1,
    'Miner': 2,
    'Phoma': 3,
    'Rust': 4
    }

# Iterate through the test data folder
for class_label in os.listdir(test_data_folder):
  class_folder = os.path.join(test_data_folder, class_label)
  true_class = class_mapping[class_label]

  # Iterate through the images in the class folder
  for image_name in os.listdir(class_folder):
    image_path = os.path.join(class_folder, image_name)

    # Load and preprocess the image, handle bad images
    img_ = load_and_preprocess_image(image_path, target_size=(256, 256))
    img = load_and_preprocess_image(image_path, target_size=(256, 256))
    if img is not None:
      true_labels.append(true_class)
      img_tensor = torch.tensor(img.transpose(2, 0, 1)).unsqueeze(0)
      with torch.no_grad():
        pred_model_07 = model_07(img_tensor)

        # Make predictions using each model
        pred_model_01 = model_01.predict(np.expand_dims(img, axis=0))
        pred_model_02 = model_02.predict(np.expand_dims(img, axis=0))
        pred_model_03 = model_03.predict(np.expand_dims(img, axis=0))
        pred_model_04 = model_04.predict(np.expand_dims(img, axis=0))
        pred_model_05 = model_05.predict(np.expand_dims(img, axis=0))
        pred_model_06 = model_06.predict(np.expand_dims(img, axis=0))
        pred_model_08 = model_08.predict(np.expand_dims(img, axis=0))

        # Append predictions to respective lists
        predictions_model_01.append(np.argmax(pred_model_01))
        predictions_model_02.append(np.argmax(pred_model_02))
        predictions_model_03.append(np.argmax(pred_model_03))
        predictions_model_04.append(np.argmax(pred_model_04))
        predictions_model_05.append(np.argmax(pred_model_05))
        predictions_model_06.append(np.argmax(pred_model_06))
        predictions_model_07.append(np.argmax(pred_model_07))
        predictions_model_08.append(np.argmax(pred_model_08))
    else:
      # Handle bad images by appending their paths to the bad_file_list
      bad_file_list.append(image_path)
      bad_count += 1

# Evaluate the model predictions (e.g., accuracy, classification report)
accuracy_model_01 = accuracy_score(true_labels, predictions_model_01)
accuracy_model_02 = accuracy_score(true_labels, predictions_model_02)
accuracy_model_03 = accuracy_score(true_labels, predictions_model_03)
accuracy_model_04 = accuracy_score(true_labels, predictions_model_04)
accuracy_model_05 = accuracy_score(true_labels, predictions_model_05)
accuracy_model_06 = accuracy_score(true_labels, predictions_model_06)
accuracy_model_07 = accuracy_score(true_labels, predictions_model_07)
accuracy_model_08 = accuracy_score(true_labels, predictions_model_08)

print("Accuracy (Model_01):", accuracy_model_01)
print("Accuracy (Model_02):", accuracy_model_02)
print("Accuracy (Model_03):", accuracy_model_03)
print("Accuracy (Model_04):", accuracy_model_04)
print("Accuracy (Model_05):", accuracy_model_05)
print("Accuracy (Model_06):", accuracy_model_06)
print("Accuracy (Model_07):", accuracy_model_07)
print("Accuracy (Model_08):", accuracy_model_08)



1/1 [==============================] - 0s 198ms/step


InvalidArgumentError: ignored

In [ ]:
# You can also generate classification reports or any other evaluation metrics as needed
report_model_01 = classification_report(true_labels, predictions_model_01)
print("Classification Report (Model_01):")
print(report_model_01)
# Repeat for other models

print(f'Number of Bad Images: {bad_count}')
print('Bad Image Paths:')
for bad_path in bad_file_list:
    print(bad_path)


Classification Report (Model_01):
              precision    recall  f1-score   support

           2       0.00      0.00      0.00        62
           3       0.00      0.00      0.00       166
           4       0.60      1.00      0.75       338

    accuracy                           0.60       566
   macro avg       0.20      0.33      0.25       566
weighted avg       0.36      0.60      0.45       566



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: ignored

#######################################################################################################

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Assuming you have loaded and named your models as model_01, model_02, etc.
# Create a list of your models
models = [
    ('model_01', model_01),
    ('model_02', model_02),
    ('model_03', model_03),
    ('model_04', model_04),
    ('model_05', model_05),
    ('model_06', model_06),
    ('model_07', model_07),
    ('model_08', model_08)
]

= "/content/drive/MyDrive/CoWorkStuff/SplitData/test"

# Create a majority voting classifier
voting_classifier = VotingClassifier(estimators=models, voting='hard')

# Assuming you have a test dataset, X_test and y_test
# Fit the ensemble model on your test data
voting_classifier.fit(X_test, y_test)

# Make predictions using the ensemble model
ensemble_predictions = voting_classifier.predict(X_test)

# Evaluate the ensemble model's performance

accuracy = accuracy_score(y_test, ensemble_predictions)
print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score

# Define the path to your test data folder
test_data_01 = "/content/drive/MyDrive/CoWorkStuff/SplitData/test"

# Initialize lists to store images and their labels
test_images = []
test_labels = []

# Create a list of your models
models = [
    ('model_01', model_01),
    ('model_02', model_02),
    ('model_03', model_03),
    ('model_04', model_04),
    ('model_05', model_05),
    ('model_06', model_06),
    ('model_07', model_07),
    ('model_08', model_08)
    ]

# Iterate through the subfolders (class labels) in your test data folder
for class_label in os.listdir(test_data_folder):
  class_folder = os.path.join(test_data_folder, class_label)

  # Iterate through image files in the class folder
  for image_file in os.listdir(class_folder):
    image_path = os.path.join(class_folder, image_file)

    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = img_array / 255.0

    # Append the preprocessed image and its class label
    test_images.append(img_array)
    test_labels.append(class_label)

# Convert the lists to NumPy arrays
X_test = np.array(test_images)
y_test = np.array(test_labels)

# Create a majority voting classifier
voting_classifier = VotingClassifier(estimators=models, voting='hard')

# Make predictions using the ensemble model
ensemble_predictions = voting_classifier.predict(X_test)

# Now you can evaluate the ensemble model's performance
accuracy = accuracy_score(y_test, ensemble_predictions)

print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')


OSError: ignored

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
import cv2

# Define the path to your test data folder
test_data_01 = "/content/drive/MyDrive/CoWorkStuff/SplitData/test"

# Initialize lists to store images and their labels
test_images = []
test_labels = []

# Initialize a list to store bad image paths
bad_file_list = []
bad_count = 0

# Create a list of your models
models = [
    ('model_01', model_01),
    ('model_02', model_02),
    ('model_03', model_03),
    ('model_04', model_04),
    ('model_05', model_05),
    ('model_06', model_06),
    ('model_07', model_07),
    ('model_08', model_08)
    ]

# Iterate through the subfolders (class labels) in your test data folder
for class_label in os.listdir(test_data_01):
  class_folder = os.path.join(test_data_01, class_label)

  # Iterate through image files in the class folder
  for image_file in os.listdir(class_folder):
    image_path = os.path.join(class_folder, image_file)

    try:
      # Load and preprocess the image
      img = load_img(image_path, target_size=(224, 224))
      img_array = img_to_array(img)
      img_array = img_array / 255.0

      # Append the preprocessed image and its class label
      test_images.append(img_array)
      test_labels.append(class_label)

    except Exception as e:
      # Handle bad images by appending their paths to the bad_file_list
      bad_file_list.append(image_path)
      bad_count += 1

# Convert the lists to NumPy arrays
X_test = np.array(test_images)
y_test = np.array(test_labels)

# Create a majority voting classifier
voting_classifier = VotingClassifier(estimators=models, voting='hard')

# Make predictions using the ensemble model
ensemble_predictions = voting_classifier.predict(X_test)

# Now you can evaluate the ensemble model's performance
accuracy = accuracy_score(y_test, ensemble_predictions)

print(f'Ensemble Model Accuracy: {accuracy * 100:.2f}%')
print(f'Number of Bad Images: {bad_count}')
print('Bad Image Paths:')
for bad_path in bad_file_list:
    print(bad_path)


KeyboardInterrupt: ignored

Example code of ensemble model to that uses the following models together for the plant disease image analysis:
• CNN
• Fast R-CNN
• GAN
• YOLO
• Image Net
• U-Net
• AlexNet
• VGG
• ResNet

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained individual models
model_cnn = load_model('cnn_model.h5')
model_fast_rcnn = load_model('fast_rcnn_model.h5')
model_gan = load_model('gan_model.h5')
model_yolo = load_model('yolo_model.h5')
model_unet = load_model('unet_model.h5')
model_alexnet = load_model('alexnet_model.h5')
model_vgg = load_model('vgg_model.h5')
model_resnet = load_model('resnet_model.h5')

# Load a majority voting classifier (or any other ensemble method of your choice)
from sklearn.ensemble import VotingClassifier

# Create an ensemble of models
ensemble_models = [
    ('cnn', model_cnn),
    ('fast_rcnn', model_fast_rcnn),
    ('gan', model_gan),
    ('yolo', model_yolo),
    ('unet', model_unet),
    ('alexnet', model_alexnet),
    ('vgg', model_vgg),
    ('resnet', model_resnet)
]

# Initialize the ensemble classifier with majority voting
ensemble = VotingClassifier(estimators=ensemble_models, voting='soft')

# Load and preprocess your test dataset
X_test, y_test = load_and_preprocess_test_data()

# Make predictions using the ensemble of models
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)

# Evaluate the ensemble model's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

report = classification_report(y_test, y_pred)
print(report)


What about the voting classifier, weighted Averaging, Stacking, Blending, Bagging and Boosting?

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained individual models
model_cnn = load_model('cnn_model.h5')
model_fast_rcnn = load_model('fast_rcnn_model.h5')
model_gan = load_model('gan_model.h5')
model_yolo = load_model('yolo_model.h5')
model_unet = load_model('unet_model.h5')
model_alexnet = load_model('alexnet_model.h5')
model_vgg = load_model('vgg_model.h5')
model_resnet = load_model('resnet_model.h5')

# Load and preprocess your test dataset
X_test, y_test = load_and_preprocess_test_data()

# Make predictions using individual models
y_pred_cnn = model_cnn.predict(X_test)
y_pred_fast_rcnn = model_fast_rcnn.predict(X_test)
y_pred_gan = model_gan.predict(X_test)
y_pred_yolo = model_yolo.predict(X_test)
y_pred_unet = model_unet.predict(X_test)
y_pred_alexnet = model_alexnet.predict(X_test)
y_pred_vgg = model_vgg.predict(X_test)
y_pred_resnet = model_resnet.predict(X_test)

# Ensemble using Weighted Averaging
weights = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]  # Adjust the weights as needed
ensemble_weighted_avg = (
    weights[0] * y_pred_cnn +
    weights[1] * y_pred_fast_rcnn +
    weights[2] * y_pred_gan +
    weights[3] * y_pred_yolo +
    weights[4] * y_pred_unet +
    weights[5] * y_pred_alexnet +
    weights[6] * y_pred_vgg +
    weights[7] * y_pred_resnet
)

# Ensemble using Stacking
from sklearn.linear_model import LogisticRegression

stacking_features = np.column_stack(
    (y_pred_cnn, y_pred_fast_rcnn, y_pred_gan, y_pred_yolo, y_pred_unet, y_pred_alexnet, y_pred_vgg, y_pred_resnet)
)

stacking_model = LogisticRegression()
stacking_model.fit(stacking_features, y_test)
y_pred_stacking = stacking_model.predict(stacking_features)

# Ensemble using Blending
blend_features = np.column_stack(
    (y_pred_cnn, y_pred_fast_rcnn, y_pred_gan, y_pred_yolo, y_pred_unet, y_pred_alexnet, y_pred_vgg, y_pred_resnet)
)

blend_model = LogisticRegression()
blend_model.fit(blend_features, y_test)
y_pred_blend = blend_model.predict(blend_features)

# Ensemble using Bagging (Bootstrap Aggregating)
from sklearn.ensemble import BaggingClassifier

bagging_classifier = BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=10)
bagging_classifier.fit(stacking_features, y_test)
y_pred_bagging = bagging_classifier.predict(stacking_features)

# Ensemble using Boosting (AdaBoost)
from sklearn.ensemble import AdaBoostClassifier

adaboost_classifier = AdaBoostClassifier(base_estimator=LogisticRegression(), n_estimators=50)
adaboost_classifier.fit(stacking_features, y_test)
y_pred_adaboost = adaboost_classifier.predict(stacking_features)

# Evaluate the ensemble models
from sklearn.metrics import accuracy_score, classification_report

print("Weighted Averaging Accuracy:", accuracy_score(y_test, ensemble_weighted_avg))
print("Stacking Accuracy:", accuracy_score(y_test, y_pred_stacking))
print("Blending Accuracy:", accuracy_score(y_test, y_pred_blend))
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))
print("Boosting Accuracy:", accuracy_score(y_test, y_pred_adaboost))

# You can also generate classification reports for detailed metrics


Show precision score of models




* Used Google Colab which was connected to "**Python 3 Google Compute Engine Backend**", pthon version 3.10.12
* The **sklearn** version used was 0.24.2.
* The **numpy** version used was 1.21.2.
*


